In [1]:
import tensorflow as tf
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from tqdm import tqdm
import os
import pandas as pd
import re
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import json

In [2]:
ontologyFile = open('./semanticOntology.json')
ontology = json.load(ontologyFile)

In [13]:
model_path = 'semanticOntology/'

In [4]:
def get_classes(node):
    classes = []
    mapping = {}
    labels = []
    if 'leaf' in node.keys():
        classes.append(node['leaf'])
    for child in node['children']:
        labels.append(child['name'])
        child_classes = get_classes(child)
        classes.extend(child_classes)
        for c in child_classes:
            mapping[c] = child['name']
    node['classes'] = classes
    node['mapping'] = mapping
    node['labels'] = labels
    return classes

get_classes(ontology)

['Arrest',
 'Assault',
 'Fighting',
 'Shooting',
 'Robbery',
 'Shoplifting',
 'Burglary',
 'Stealing',
 'Explosion',
 'Arson',
 'Vandalism',
 'RoadAccidents']

In [5]:
train_data_dir = './data/Train'
test_data_dir = './data/Val'

folds = 5
test_fold = 0
if os.path.exists(train_data_dir):
    shutil.rmtree(train_data_dir)
if os.path.exists(test_data_dir):
    shutil.rmtree(test_data_dir)

for category in tqdm(get_classes(ontology)):
    os.makedirs(train_data_dir + '/' + category + '/')
    for fold in [x for x in range(folds) if x != test_fold]:
        files = os.listdir('./data/' + str(fold) + '/' + category + '/')
        files = [x for x in files if x != 'desktop.ini']
        for filename in files:
            shutil.copy('./data/' + str(fold) + '/' + category + '/' + filename, train_data_dir + '/' + category + '/')

    os.makedirs(test_data_dir + '/' + category + '/')
    files = os.listdir('./data/' + str(test_fold) + '/' + category + '/')
    files = [x for x in files if x != 'desktop.ini']
    for filename in files:
        shutil.copy('./data/' + str(test_fold) + '/' + category + '/' + filename, test_data_dir + '/' + category + '/')

100%|██████████| 12/12 [00:34<00:00,  2.91s/it]


In [8]:
img_height, img_width = 240, 320
batch_size = 64

def preprocess(images, labels):
    return tf.keras.applications.resnet50.preprocess_input(images), labels

print('Loading dataset')
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    seed=0,
    label_mode='categorical',
    image_size=(img_height, img_width),
    batch_size=None)

class_names = train_dataset.class_names

print('Preprocessing')
train_dataset = train_dataset.map(preprocess)

Loading dataset
Found 7250 files belonging to 12 classes.
Preprocessing


In [7]:
def train_nodes(node):
    print(f'Training {node["name"]}')
    n_classes = len(node['children'])

    def filter_classes(image, label):
        return tf.py_function(func=lambda y: y in node['classes'], inp=[tf.gather(class_names, tf.math.argmax(label))], Tout=tf.bool)

    def relabel(image, label):
        new_label = tf.py_function(func=lambda y: node['labels'].index(node['mapping'][y.numpy().decode()]), inp=[tf.gather(class_names, tf.math.argmax(label))], Tout=tf.int64)
        new_label = tf.one_hot(new_label, depth=n_classes)
        new_label.set_shape([n_classes])
        return image, new_label


    train_ds = train_dataset.filter(filter_classes)
    train_ds = train_ds.map(relabel)
    train_ds = train_ds.batch(batch_size)

    model = Sequential()
    resnet = tf.keras.applications.ResNet50(
        include_top=False,
        input_shape=(img_height, img_width, 3),
        pooling='avg',
        weights='imagenet'
    )
    resnet.trainable = False
    model.add(resnet)
    model.add(Dense(512, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_ds, epochs=3)

    model.save(model_path + node["name"])

    for c in node['children']:
        if 'leaf' not in c.keys():
            train_nodes(c)

train_nodes(ontology)

Training dataset
Epoch 1/3
114/114 [==============================] - 34s 262ms/step - loss: 2.8208 - accuracy: 0.6321
Epoch 2/3
114/114 [==============================] - 29s 250ms/step - loss: 0.4825 - accuracy: 0.8142
Epoch 3/3
114/114 [==============================] - 29s 253ms/step - loss: 0.2751 - accuracy: 0.9054


INFO:tensorflow:Assets written to: ./models/semanticOntology/dataset\assets


INFO:tensorflow:Assets written to: ./models/semanticOntology/dataset\assets


Training attack
Epoch 1/3
31/31 [==============================] - 12s 351ms/step - loss: 5.3432 - accuracy: 0.5291
Epoch 2/3
31/31 [==============================] - 12s 366ms/step - loss: 0.5725 - accuracy: 0.7806
Epoch 3/3
31/31 [==============================] - 12s 385ms/step - loss: 0.3314 - accuracy: 0.8918


INFO:tensorflow:Assets written to: ./models/semanticOntology/attack\assets


...

INFO:tensorflow:Assets written to: ./models/semanticOntology/property_damage\assets


INFO:tensorflow:Assets written to: ./models/semanticOntology/property_damage\assets


Training fire
Epoch 1/3
13/13 [==============================] - 9s 554ms/step - loss: 9.5232 - accuracy: 0.5720
Epoch 2/3
13/13 [==============================] - 7s 483ms/step - loss: 0.4308 - accuracy: 0.8183
Epoch 3/3
13/13 [==============================] - 7s 488ms/step - loss: 0.2067 - accuracy: 0.9171


INFO:tensorflow:Assets written to: ./models/semanticOntology/fire\assets


INFO:tensorflow:Assets written to: ./models/semanticOntology/fire\assets


In [6]:
test_data_dir = './data/Val'
img_height, img_width = 240, 320

categories = get_classes(ontology)

test_df = pd.DataFrame()
for category in tqdm(categories):
    images = os.listdir(test_data_dir + '/' + category)
    images = [test_data_dir + '/' + category + '/' + x for x in images if x != 'desktop.ini']
    df = pd.DataFrame({'image': images})
    df['recording'] = df['image'].map(lambda x: re.search(r'.*/([a-zA-X]+\d+)_.*\.jpg', x).group(1))
    df['class'] = category
    test_df = pd.concat([test_df, df], ignore_index=True)

def preprocess_image(image):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [img_height, img_width])
    image = tf.keras.applications.resnet50.preprocess_input(image)
    image = image.numpy()
    return image

test_df['image'] = [preprocess_image(x) for x in test_df['image']]


100%|██████████| 12/12 [00:00<00:00, 773.38it/s]


In [14]:
prediction_dict = {}
for category in categories:
    prediction_dict[category] = 1
test_df['prediction'] = [prediction_dict.copy() for _ in range(len(test_df))]

test_ds = tf.data.Dataset.from_generator(lambda: test_df['image'], output_signature=(tf.TensorSpec(shape=(240, 320, 3), dtype=tf.float32)))
test_ds = test_ds.batch(batch_size)

def test_nodes(node):
    print(f'Testing {node["name"]}')

    model = tf.keras.models.load_model("./models/" + str(test_fold) + "/" + model_path + node['name'])
    prediction = model.predict(test_ds)

    def prediction_mapping(prediction):
        classification = {}
        for category, label in node['mapping'].items():
            classification[category] = prediction[node['labels'].index(label)]
        return classification

    predictions = [prediction_mapping(x) for x in prediction]

    def update_predictions(prediction):
        for category in node['classes']:
            prediction['prediction'][category] *= predictions[prediction.name][category]

    test_df.apply(update_predictions, axis=1)
    for c in node['children']:
        if 'leaf' not in c.keys():
            test_nodes(c)

test_nodes(ontology)

Testing dataset
30/30 [==============================] - 9s 203ms/step
Testing attack
30/30 [==============================] - 6s 198ms/step
Testing non_weaponized
30/30 [==============================] - 6s 200ms/step
Testing theft
30/30 [==============================] - 6s 201ms/step
Testing non_violent
30/30 [==============================] - 6s 200ms/step
Testing property_damage
30/30 [==============================] - 6s 200ms/step
Testing fire
30/30 [==============================] - 6s 199ms/step


In [49]:
def sum_predictions(predictions):
    final_prediction = {}
    for category in categories:
        final_prediction[category] = 0
        for prediction in predictions:
            final_prediction[category] += prediction[category]
    return final_prediction

predictions = test_df.groupby('recording').agg({'class': 'first', 'prediction': sum_predictions})
predictions['prediction'] = predictions['prediction'].map(lambda x: max(x, key=x.get))

def find_ancesotrs(prediction, ontology, pred, true):
    for c in ontology["children"]:
        if prediction["prediction"] in c["classes"]:
            pred.append(c["name"])
        if prediction["class"] in c["classes"]:
            true.append(c["name"])
        find_ancesotrs(prediction, c, pred, true)

def add_ancestors(prediction):
    print(prediction)
    pred = []
    true = []
    find_ancesotrs(prediction, ontology, pred, true)
    prediction["pred_ancestors"] = pred
    prediction["true_ancestors"] = true

add_ancestors(predictions.iloc[0])
# print(classification_report(predictions['class'], predictions['prediction']))

class         Assault
prediction    Assault
Name: Abuse003, dtype: object
['attack', 'non_weaponized', 'assault'] ['attack', 'non_weaponized', 'assault']
